# Hashtable

The 'Dictionary-approach' that was used in the previous exercise works very nicely. Python actually makes a hashtable (or hash map) of it, where the flow IDs and associated flow size are stored as key-value pairs. In HDL, there are no such concepts like hashtables, so the hash table has to be implemented manually.

When a FlowID comes in, it is first hashed. For the sake of completeness it is mentioned that **a hash function** is a function that transforms an input of an arbitrary length to an output of a fixed length, called the **digest** (or hash).

The digest is subsequently used as an address to a memory. The value that resides on that memory address is the **value**, where the FlowID is the **key**. 

<center>
<img src="images/21_hashtable.png"/>
</center>

If we want to reduce the memory footprint, we can hash the flowids to locate an index to store the flowids and flowsize. 

## How to address hash collisions?

When a hash function provides a 256 bits hash, this would imply that the memory is 2<sup>256</sup> deep. As this is barely feasible, only of subset of a hash function (or even a tiny hash function) is used.

Since we are trimming down the hash according to the length of the hash table, there will be **collisions**.

> A **collision** occurs when two different FlowIDs point to the same value as the (part of the) hash digest is identical.

Let's try to mimic the hardware behaviour an try to (re-)build a hashtable. For hash algorithm the MD5 algorithms is chosen. This function generates a 128-bit digest. 

To keep the resources usage under control, only the 8 least significant bits of the hash digest are considered.

In [14]:
import hashlib

HASHTABLE_DEPTH = 4096 # depth of the hashtable
memory = [None] * HASHTABLE_DEPTH

def _hash(key):
    """ Md5 hash function to calculate the index"""
    HASHTABLE_DEPTH = 4096
    md5 = hashlib.md5(str(hash(key)).encode('utf-8'))
    return int(md5.hexdigest(), 16) % HASHTABLE_DEPTH
    
def add_ht(key, value):
    # write to code to obtain a numeric hash value for the 'key'
    index = _hash(key)

    # write the code to check if a value already exists on the obtained address
    # if the address is not set yet, set it to the value
    # if the address is already set, overwrite it with the sum of both values
    if(memory[index]) is not None:
        memory[index] += value
    else:
        memory[index] = value
        
def query_ht(key):
    # write to code to obtain a numeric hash value for the 'key'
    index = _hash(key)

    # return zero (0x0) if no entry exists
    # if an entry exists, return the corresponding value
    if memory[index] is None:
        return 0
    else:
        return memory[index]

# Set a 'key'
key = "the quick brown fox jumps over the lazy dog"

# Query the 'key'
queried_value = query_ht(key)
print("%s => %d" % (key, queried_value))
        
# Add the 'key'
add_ht(key, 123)
queried_value = query_ht(key)
print("%s => %d" % (key, queried_value))

# Add the 'key'
add_ht(key, 321)
queried_value = query_ht(key)
print("%s => %d" % (key, queried_value))


the quick brown fox jumps over the lazy dog => 0
the quick brown fox jumps over the lazy dog => 123
the quick brown fox jumps over the lazy dog => 444


<center><div style="background-color: #10FF107f;">The code above should report 0, 123, and 444.</div></center>

Now, lets run the exercise using the **dataset**. Loop through the dataset again and determine which of the flowids exhibits anomalous behaviour by exceeding the allocated bandwidth. However, this time use the self-fabricated hashtable.

**Note**: This time we are going to extract the actual flow volumes from the dataset

(**HINT**: you can recover most of the work from your previous exercises.)

In [15]:
from lib.dataset import NIDSDataset

data_file = 'data/dataset_packets_v2.npy'
labels_file = 'data/dataset_labels_v1.npy'

dataset = NIDSDataset(data_file, labels_file)

In [16]:
""" Now, reading the dataset again to update the hashtable. you can Copy and """
""" paste the code from the previous Exercise. """
""" In this exercise we will be taking the actual flow volume instead of flow size = 1 """
memory = [None] * HASHTABLE_DEPTH
wordcounter=0
flowid = ""     # flow id
flowvolume = "" # flow volume (Total length from the ip header)
flowlist = []  # keeps a list to store flows

# loop over all datasets
for d in dataset:

    decision_is_made = 0 # decision_is_made = 1 when ethertype is not 0x0800 or packet is neither TCP nor UDP
    wordcounter = 0
    flowid = ""
    flowvolume = "" 
    flowid_complete = 0

    # loop over all words
    for word in d:
        # stop parsing if a decission is made
        if decision_is_made == 0:

            # examine if Ethertype is 0x0800 - in link layer header
            # if Ethertype is not equal to 0x0800, break loop
            """WRITE code here"""
            if wordcounter == 3:
                if(word[0] == 8) and (word[1] == 0):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
            
            # examine if proto is tcp or udp 6/17 - in network layer header
            # if proto is not tcp or udp, break loop
            """WRITE code here"""
            if wordcounter == 5:
                if(word[3] == 6)or(word[3] == 17):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
            
                    
            # extract Total length (flow volume) - in network layer header
            # hint: convert to hex and concatanate the bytes
            """WRITE code here"""
            if(wordcounter == 4):
                flowvolume += hex(word[0])[2:] # ip len 1/2
                flowvolume += hex(word[1])[2:]  # ip len 2/2
            
                
            # Extract flowid. flowid is (sorce address, dest address, source port, dest port)
            # hint: convert to hex and remove 0x. concatanate the addresses and ports
            
            # extract Source Address - in network layer header
            """WRITE code here"""
            if wordcounter == 6:
                flowid += hex(word[2])[2:]  # ip SA 1/4
                flowid += hex(word[3])[2:]  # ip SA 2/4
            if wordcounter == 7:
                flowid += hex(word[0])[2:]  # ip SA 3/4
                flowid += hex(word[1])[2:]  # ip SA 4/4

            # extract Destination Address - in network layer header
            """WRITE code here"""
            if wordcounter == 7:
                flowid += hex(word[2])[2:]  # ip DA 1/4
                flowid += hex(word[3])[2:]  # ip DA 2/4
            if wordcounter == 8:
                flowid += hex(word[0])[2:]  # ip DA 3/4
                flowid += hex(word[1])[2:]  # ip DA 4/4
            
            # extract source port estination port - in network layer header
            """WRITE code here"""
            if wordcounter == 8:
                flowid += hex(word[2])[2:]  # ip SPort 1/2
                flowid += hex(word[3])[2:]  # ip SPort 2/2
            

            # examine Destination port - in transport layer header
            # If the flowid is complete, set the flag flowid_complete to 1 and break out of the loop
            """ WRITE code here """
            if wordcounter == 9:
                flowid += hex(word[0])[2:]  # ip DPort 1/2
                flowid += hex(word[1])[2:]  # ip DPort 2/2
                flowid_complete = 1
            
            
        wordcounter += 1
        
    if(flowid_complete == 1): 
        if flowid not in flowlist:
            flowlist.append(flowid)
            
        # Convert the flow volume to integer
        """ WRITE code here """
        flowvolume = (int("0x"+str(flowvolume),16))

        """ Updating the table """
        # add flowid and flowvolume to the hashtable using the function
        """ WRITE code here """
        if(flowid_complete == 1):
            add_ht(flowid,flowvolume)
        

"""Print the flow IDs that exceeds a threshold"""
threshold = 1000

# iterate through the flowlist and print those flowids having total volume greater than the threshold
count_flows = 0
count_malicious = 0
"""WRITE code here """
for flowid in flowlist:
    count_flows += 1
    volume = query_ht(flowid)
    if(volume>threshold):
        print(flowid, '->', volume)
        count_malicious += 1

print("Total flows = ",count_flows)
print("Number of malicious flows = ", count_malicious)


c0a8a32c0a8a3db2ccc4 -> 14560
c0a8a3c0a8a32cc4db2c -> 1984
c0a8a32c0a8a3a4a0185 -> 14560
c0a8a3c0a8a32185a4a0 -> 5888
c0a8a9e000fcf6ea14eb -> 1232
c0a8a9c0a8a345c27 -> 1092
c0a8a9c0a8a3089089 -> 1920
c0a8a3c0a8a9089089 -> 1800
c0a8a9c0a8a3461bd -> 1300
c0a8a3c0a8a91bd46 -> 1040
c0a8a19c0a8aff08a08a -> 11712
c0a8a32c0a8aff08a08a -> 1150
c0a8a19c0a8a3089089 -> 2880
c0a8a3c0a8a19089089 -> 2700
c0a8a19e000fb14e914e9 -> 3024
Total flows =  195
Number of malicious flows =  15


<center><div style="background-color: #10FF107f;">The code above should report that there are 195 FlowIDs in the dataset, of which 15 exceed the allowed bandwidth.</div></center>


## Chaining

In order to handle possible collisions when hashing, measures have to be taken. **Chaining** is a simple solution and is like a linked list. Each each index can include a separate list with many elements. The advantage is that hash table never fills up and we can always add more elements to the chain.

<center>
<img src="images/counting/chaining.png"/>
</center>

However, linked lists are not tailored for hardware. Also, is required to store the flowids which drastically increases the memory footprint. Hence collision resistant data structures such as sketches and counting bloom filters are usually employed in hardware implementations.


### Optional exercise on chaining

The below given exercise is optional. If time permits, you can try out the same exercise given in the previous exercise using chaining in the cell below.

In [17]:
import hashlib

HASHTABLE_DEPTH = 4096 # depth of the hashtable
memory = [None] * HASHTABLE_DEPTH

def _hash(key):
    """ Md5 hash function to calculate the index"""
    HASHTABLE_DEPTH = 4096
    md5 = hashlib.md5(str(hash(key)).encode('utf-8'))
    return int(md5.hexdigest(), 16) % HASHTABLE_DEPTH
    
def add_ht(key, value):
    # write to code to obtain a numeric hash value for the 'key'
    index = _hash(key)

    # write the code to check if a value already exists on the obtained address
    # if the address is not set yet, set it to the value
    # if the address is already set, overwrite it with the sum of both values
    if memory[index] is not None:  # This index already contain some values.
        # WRITE the code to Check if the flowid present in the [key,value] pair in the array[index] is equal to 
        # the incoming flowid. if equal, then add the flow size to the value.
        # If the flowids are not equal, then we have to think of the chaining and append the new
        # element to the list in the array[index].
        for kvp in memory[index]:  
            if kvp[0] == flowid:  
                kvp[1] = kvp[1]+value 
                break 
        else:     
            memory[index].append([flowid, value])  
    else: 
        # WRITE the code here
        # If the index is empty, creare an empty list in the array[index] and append the key-value pair to the list.
        memory[index] = []
        memory[index].append([flowid, value])
        
def query_ht(key):
    # write to code to obtain a numeric hash value for the 'key'
    index = _hash(key)

    # return zero (0x0) if no entry exists
    # if an entry exists, return the corresponding value
    if memory[index] is None:
        return 0
    else:
        """ WRITE code """
        # Loop through all key-value-pairs and find if the flowid exist. 
        # If exists then return its value. # If no return was done during loop, 
        # it means flowid does not exist and return 0
        for kvp in memory[index]:
            if kvp[0] == key:
                return kvp[1]
        return 0


##### Reading the data set and updating the hashtable

In [18]:
""" Now, reading the dataset again to update the hashtable. you can Copy and """
""" paste the code from the previous Exercise. """
""" In this exercise we will be taking the actual flow volume instead of flow size = 1 """
memory = [None] * HASHTABLE_DEPTH
wordcounter=0
flowid = ""     # flow id
flowvolume = "" # flow volume (Total length from the ip header)
flowlist = []  # keeps a list to store flows

# loop over all datasets
for d in dataset:

    decision_is_made = 0 # decision_is_made = 1 when ethertype is not 0x0800 or packet is neither TCP nor UDP
    wordcounter = 0
    flowid = ""
    flowvolume = "" 
    flowid_complete = 0

    # loop over all words
    for word in d:
        # stop parsing if a decission is made
        if decision_is_made == 0:

            # examine if Ethertype is 0x0800 - in link layer header
            # if Ethertype is not equal to 0x0800, break loop
            """WRITE code here"""
            if wordcounter == 3:
                if(word[0] == 8) and (word[1] == 0):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
            
            # examine if proto is tcp or udp 6/17 - in network layer header
            # if proto is not tcp or udp, break loop
            """WRITE code here"""
            if wordcounter == 5:
                if(word[3] == 6)or(word[3] == 17):
                    decision_is_made = 0
                else:
                    decision_is_made = 1
                    break
            
                    
            # extract Total length (flow volume) - in network layer header
            # hint: convert to hex and concatanate the bytes
            """WRITE code here"""
            if(wordcounter == 4):
                flowvolume += hex(word[0])[2:] # ip len 1/2
                flowvolume += hex(word[1])[2:]  # ip len 2/2
            
                
            # Extract flowid. flowid is (sorce address, dest address, source port, dest port)
            # hint: convert to hex and remove 0x. concatanate the addresses and ports
            
            # extract Source Address - in network layer header
            """WRITE code here"""
            if wordcounter == 6:
                flowid += hex(word[2])[2:]  # ip SA 1/4
                flowid += hex(word[3])[2:]  # ip SA 2/4
            if wordcounter == 7:
                flowid += hex(word[0])[2:]  # ip SA 3/4
                flowid += hex(word[1])[2:]  # ip SA 4/4

            # extract Destination Address - in network layer header
            """WRITE code here"""
            if wordcounter == 7:
                flowid += hex(word[2])[2:]  # ip DA 1/4
                flowid += hex(word[3])[2:]  # ip DA 2/4
            if wordcounter == 8:
                flowid += hex(word[0])[2:]  # ip DA 3/4
                flowid += hex(word[1])[2:]  # ip DA 4/4
            
            # extract source port estination port - in network layer header
            """WRITE code here"""
            if wordcounter == 8:
                flowid += hex(word[2])[2:]  # ip SPort 1/2
                flowid += hex(word[3])[2:]  # ip SPort 2/2
            

            # examine Destination port - in transport layer header
            # If the flowid is complete, set the flag flowid_complete to 1 and break out of the loop
            """ WRITE code here """
            if wordcounter == 9:
                flowid += hex(word[0])[2:]  # ip DPort 1/2
                flowid += hex(word[1])[2:]  # ip DPort 2/2
                flowid_complete = 1
            
            
        wordcounter += 1
        
    if(flowid_complete == 1): 
        if flowid not in flowlist:
            flowlist.append(flowid)
            
        # Convert the flow volume to integer
        """ WRITE code here """
        flowvolume = (int("0x"+str(flowvolume),16))

        """ Updating the table """
        # add flowid and flowvolume to the hashtable using the function
        """ WRITE code here """
        if(flowid_complete == 1):
            add_ht(flowid,flowvolume)
        

"""Print the flow IDs that exceeds a threshold"""
threshold = 1000

# iterate through the flowlist and print those flowids having total volume greater than the threshold
count_flows = 0
count_malicious = 0
"""WRITE code here """
for flowid in flowlist:
    count_flows += 1
    volume = query_ht(flowid)
    if(volume>threshold):
        print(flowid, '->', volume)
        count_malicious += 1

print("Total flows = ",count_flows)
print("Number of malicious flows = ", count_malicious)


c0a8a32c0a8a3db2ccc4 -> 14560
c0a8a3c0a8a32cc4db2c -> 1984
c0a8a32c0a8a3a4a0185 -> 14560
c0a8a3c0a8a32185a4a0 -> 5888
c0a8a9e000fcf6ea14eb -> 1232
c0a8a9c0a8a345c27 -> 1092
c0a8a9c0a8a3089089 -> 1920
c0a8a3c0a8a9089089 -> 1800
c0a8a9c0a8a3461bd -> 1300
c0a8a3c0a8a91bd46 -> 1040
c0a8a19c0a8aff08a08a -> 11712
c0a8a32c0a8aff08a08a -> 1150
c0a8a19c0a8a3089089 -> 2880
c0a8a3c0a8a19089089 -> 2700
c0a8a19e000fb14e914e9 -> 3024
Total flows =  195
Number of malicious flows =  15


<center><div style="background-color: #10FF107f;">The code above should report that there are 195 FlowIDs in the dataset, of which 15 exceed the allowed bandwidth.</div></center>

<hr/>
<center>
Continue with the <a href="22_counting.ipynb">next notebook</a> in a new browser tab.<br/><br/>
<img src="images/footer.png"/>
</center>